In [21]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText

# Constants
ARBITRAGE_THRESHOLD = 30  # Define your threshold
DATABASE_URL = 'mongodb://localhost:27017/'  # MongoDB URL
DATABASE_NAME = 'bigdataproject'  
COLLECTION_NAME = 'trades'  

In [22]:
import getpass

# Email constants
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587  # For SSL
SENDER_EMAIL = 'bigdataarbitrage@gmail.com'  # Enter your address
RECEIVER_EMAIL = 'bigdataarbitrage@gmail.com'  # Enter receiver address

# Prompt for the sender's email password securely
print("Please enter your email password:")
SENDER_PASSWORD = getpass.getpass()



Please enter your email password:


 ········


In [19]:
import time
import pandas as pd
from pymongo import MongoClient
import smtplib
from email.mime.text import MIMEText


# Function to send email
def send_email(subject, content):
    message = MIMEText(content)
    message['From'] = SENDER_EMAIL
    message['To'] = RECEIVER_EMAIL
    message['Subject'] = subject

    server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    server.starttls()
    server.login(SENDER_EMAIL, SENDER_PASSWORD)
    server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message.as_string())
    server.quit()

# Function to process recent data for arbitrage opportunities
def process_opportunities():
    with MongoClient(DATABASE_URL) as client:
        db = client[DATABASE_NAME]
        collection = db[COLLECTION_NAME]

        # Get the most recent document to determine the latest 'time_exchange'
        latest_document = collection.find_one({}, sort=[('time_exchange', -1)])
        if not latest_document:
            print("No trades found in the database.")
            return

        latest_time = latest_document['time_exchange']
        ten_minutes_ago = latest_time - 10 * 60 * 1000  # Assuming 'time_exchange' is in milliseconds

        # Fetch documents from the last 10 minutes based on the latest time_exchange
        recent_trades = list(collection.find({'time_exchange': {'$gte': ten_minutes_ago, '$lte': latest_time}}))

        if not recent_trades:
            print("No trades found in the last 10 minutes from the most recent data.")
            return

        # Convert to DataFrame for easier manipulation
        recent_data = pd.DataFrame(recent_trades)

        # Group data by 'symbol_id' to calculate arbitrage opportunities per symbol
        for symbol_id, group in recent_data.groupby('symbol_id'):
            min_buy_price = group['min_buy_price'].min()
            max_sell_price = group['max_sell_price'].max()
            price_diff = max_sell_price - min_buy_price

            if price_diff > 0:
                # Find the trades corresponding to the min buy price and max sell price
                buy_trade = group[group['min_buy_price'] == min_buy_price].iloc[0]
                sell_trade = group[group['max_sell_price'] == max_sell_price].iloc[0]

                # Construct the email content
                email_subject = f"Arbitrage Opportunity Detected - {symbol_id}"
                email_content = (
                    f"Symbol: {symbol_id}\n"
                    f"Time: Buy at {buy_trade['time_exchange']}, Sell at {sell_trade['time_exchange']}\n"
                    f"Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}\n"
                    f"Price Difference: {price_diff}\n"
                    f"Buy Trade Details: {buy_trade.to_dict()}\n"
                    f"Sell Trade Details: {sell_trade.to_dict()}\n"
                )
                send_email(email_subject, email_content)

# Main loop to run the process every 10 minutes
while True:
    process_opportunities()
    time.sleep(600)  # Wait 10 minutes before the next check


No trades found in the database.


KeyboardInterrupt: 

### Final Email alert:

In [4]:
import time
import pandas as pd
from pymongo import MongoClient
import smtplib
from email.mime.text import MIMEText

# Function to send email
def send_email(subject, content):
    message = MIMEText(content)
    message['From'] = SENDER_EMAIL
    message['To'] = RECEIVER_EMAIL
    message['Subject'] = subject

    server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    server.starttls()
    server.login(SENDER_EMAIL, SENDER_PASSWORD)
    server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message.as_string())
    server.quit()

# Function to process recent data for arbitrage opportunities
def process_opportunities():
    with MongoClient(DATABASE_URL) as client:
        db = client[DATABASE_NAME]
        collection = db[COLLECTION_NAME]

        # Get the most recent document to determine the latest 'time_exchange'
        latest_document = collection.find_one({}, sort=[('time_exchange', -1)])
        if not latest_document:
            print("No trades found in the database.")
            return

        latest_time = latest_document['time_exchange']
        print(f"Latest time: {latest_time}")  # Debug statement

        ten_minutes_ago = latest_time - 10 * 60 * 1000  # Assuming 'time_exchange' is in milliseconds
        print(f"Checking trades from {ten_minutes_ago} to {latest_time}")  # Debug statement

        # Fetch documents from the last 10 minutes based on the latest time_exchange
        recent_trades = list(collection.find({'time_exchange': {'$gte': ten_minutes_ago, '$lte': latest_time}}))
        print(f"Found {len(recent_trades)} trades in the last 10 minutes.")  # Debug statement

        if not recent_trades:
            print("No trades found in the last 10 minutes from the most recent data.")
            return

        # Convert to DataFrame for easier manipulation
        recent_data = pd.DataFrame(recent_trades)

        # Group data by 'symbol_id' to calculate arbitrage opportunities per symbol
        for symbol_id, group in recent_data.groupby('symbol_id'):
            print(f"Processing {symbol_id} with {len(group)} trades.")  # Debug statement

            min_buy_price = group['price'].min()  # Assuming 'price' is used directly
            max_sell_price = group['price'].max()
            price_diff = max_sell_price - min_buy_price

            print(f"{symbol_id} - Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}, Price Difference: {price_diff}")  # Debug statement

            if price_diff > 0:
                # Find the trades corresponding to the min buy price and max sell price
                buy_trade = group[group['price'] == min_buy_price].iloc[0]
                sell_trade = group[group['price'] == max_sell_price].iloc[0]

                # Construct the email content
                email_subject = f"Arbitrage Opportunity Detected - {symbol_id}"
                email_content = (
                    f"Symbol: {symbol_id}\n"
                    f"Time: Buy at {buy_trade['time_exchange']}, Sell at {sell_trade['time_exchange']}\n"
                    f"Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}\n"
                    f"Price Difference: {price_diff}\n"
                    f"Buy Trade Details: {buy_trade.to_dict()}\n"
                    f"Sell Trade Details: {sell_trade.to_dict()}\n"
                )
                send_email(email_subject, email_content)

# Main loop to run the process every 10 minutes
while True:
    process_opportunities()
    time.sleep(600)  # Wait 10 minutes before the next check


Latest time: 1714219444928
Checking trades from 1714218844928 to 1714219444928
Found 27 trades in the last 10 minutes.
Processing BITFINEX_SPOT_BTC_USD with 8 trades.
BITFINEX_SPOT_BTC_USD - Min Buy Price: 63348.0, Max Sell Price: 63508.0, Price Difference: 160.0
Processing BITFINEX_SPOT_BTC_USDT with 10 trades.
BITFINEX_SPOT_BTC_USDT - Min Buy Price: 63366.0, Max Sell Price: 63403.0, Price Difference: 37.0
Processing BITSTAMP_SPOT_BTC_USD with 4 trades.
BITSTAMP_SPOT_BTC_USD - Min Buy Price: 63307.0, Max Sell Price: 63357.0, Price Difference: 50.0
Processing KRAKEN_SPOT_BTC_USD with 1 trades.
KRAKEN_SPOT_BTC_USD - Min Buy Price: 63269.3, Max Sell Price: 63269.3, Price Difference: 0.0
Processing KRAKEN_SPOT_BTC_USDT with 4 trades.
KRAKEN_SPOT_BTC_USDT - Min Buy Price: 63360.0, Max Sell Price: 63360.0, Price Difference: 0.0


KeyboardInterrupt: 

In [20]:
import time
import pandas as pd
from pymongo import MongoClient
import smtplib
from email.mime.text import MIMEText

# Function to send email
def send_email(subject, content):
    message = MIMEText(content)
    message['From'] = SENDER_EMAIL
    message['To'] = RECEIVER_EMAIL
    message['Subject'] = subject

    server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    server.starttls()
    server.login(SENDER_EMAIL, SENDER_PASSWORD)
    server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message.as_string())
    server.quit()

# Function to process recent data for arbitrage opportunities
def process_opportunities():
    with MongoClient(DATABASE_URL) as client:
        db = client[DATABASE_NAME]
        collection = db[COLLECTION_NAME]

        # Get the most recent document to determine the latest 'time_exchange'
        latest_document = collection.find_one({}, sort=[('time_exchange', -1)])
        if not latest_document:
            print("No trades found in the database.")
            return

        latest_time = latest_document['time_exchange']
        print(f"Latest time: {latest_time}")  # Debug statement

        ten_minutes_ago = latest_time - 10 * 60 * 1000  # Assuming 'time_exchange' is in milliseconds
        print(f"Checking trades from {ten_minutes_ago} to {latest_time}")  # Debug statement

        # Fetch documents from the last 10 minutes based on the latest time_exchange
        recent_trades = list(collection.find({'time_exchange': {'$gte': ten_minutes_ago, '$lte': latest_time}}))
        print(f"Found {len(recent_trades)} trades in the last 10 minutes.")  # Debug statement

        if not recent_trades:
            print("No trades found in the last 10 minutes from the most recent data.")
            return

        # Convert to DataFrame for easier manipulation
        recent_data = pd.DataFrame(recent_trades)
        # Exclude trades with a 'min_buy_price' of zero
        recent_data = recent_data[recent_data['min_buy_price'] > 0]

        # Calculate arbitrage opportunities using specific columns
        min_buy_price = recent_data['min_buy_price'].min()
        max_sell_price = recent_data['max_sell_price'].max()
        price_diff = max_sell_price - min_buy_price

        # Calculate arbitrage opportunities across all symbols
        #min_buy_price = recent_data['price'].min()
        #max_sell_price = recent_data['price'].max()
        #price_diff = max_sell_price - min_buy_price

        print(f"All Symbols - Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}, Price Difference: {price_diff}")

        if price_diff > 0:
            # Find the trades corresponding to the min buy price and max sell price
            buy_trade = recent_data[recent_data['price'] == min_buy_price].iloc[0]
            sell_trade = recent_data[recent_data['price'] == max_sell_price].iloc[0]

            # Construct the email content
            email_subject = "Arbitrage Opportunity Detected Across All Symbols"
            email_content = (
                f"Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}\n"
                f"Price Difference: {price_diff}\n"
                f"Buy Trade Details: {buy_trade.to_dict()}\n"
                f"Sell Trade Details: {sell_trade.to_dict()}\n"
            )
            send_email(email_subject, email_content)

# Main loop to run the process every 10 minutes
while True:
    process_opportunities()
    time.sleep(600)  # Wait 10 minutes before the next check


Latest time: 1714232856401
Checking trades from 1714232256401 to 1714232856401
Found 20 trades in the last 10 minutes.
All Symbols - Min Buy Price: 62920.9, Max Sell Price: 63652.0, Price Difference: 731.0999999999985


IndexError: single positional indexer is out-of-bounds

In [23]:
import time
import pandas as pd
from pymongo import MongoClient
import smtplib
from email.mime.text import MIMEText

# Function to send email
def send_email(subject, content):
    message = MIMEText(content)
    message['From'] = SENDER_EMAIL
    message['To'] = RECEIVER_EMAIL
    message['Subject'] = subject

    server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    server.starttls()
    server.login(SENDER_EMAIL, SENDER_PASSWORD)
    server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message.as_string())
    server.quit()

# Function to process recent data for arbitrage opportunities
def process_opportunities():
    with MongoClient(DATABASE_URL) as client:
        db = client[DATABASE_NAME]
        collection = db[COLLECTION_NAME]

        # Get the most recent document to determine the latest 'time_exchange'
        latest_document = collection.find_one({}, sort=[('time_exchange', -1)])
        if not latest_document:
            print("No trades found in the database.")
            return

        latest_time = latest_document['time_exchange']
        print(f"Latest time: {latest_time}")  # Debug statement

        ten_minutes_ago = latest_time - 10 * 60 * 1000  # Assuming 'time_exchange' is in milliseconds
        print(f"Checking trades from {ten_minutes_ago} to {latest_time}")  # Debug statement

        # Fetch documents from the last 10 minutes based on the latest time_exchange
        recent_trades = list(collection.find({'time_exchange': {'$gte': ten_minutes_ago, '$lte': latest_time}}))
        print(f"Found {len(recent_trades)} trades in the last 10 minutes.")  # Debug statement

        if not recent_trades:
            print("No trades found in the last 10 minutes from the most recent data.")
            return

        # Convert to DataFrame for easier manipulation
        recent_data = pd.DataFrame(recent_trades)
        # Exclude trades with a 'min_buy_price' of zero
        recent_data = recent_data[recent_data['min_buy_price'] > 0]

        # Calculate arbitrage opportunities using specific columns
        min_buy_price = recent_data['min_buy_price'].min()
        max_sell_price = recent_data['max_sell_price'].max()
        price_diff = max_sell_price - min_buy_price

        # Calculate arbitrage opportunities across all symbols
        #min_buy_price = recent_data['price'].min()
        #max_sell_price = recent_data['price'].max()
        #price_diff = max_sell_price - min_buy_price

        print(f"All Symbols - Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}, Price Difference: {price_diff}")

        if price_diff > 0:
            # Find the trades corresponding to the min buy price and max sell price
            buy_trade = recent_data[recent_data['price'] == min_buy_price].iloc[0]
            sell_trade = recent_data[recent_data['price'] == max_sell_price].iloc[0]
            

            # Construct the email content
            email_subject = "Arbitrage Opportunity Detected Across All Symbols"
            email_content = (
                f"Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}\n"
                f"Price Difference: {price_diff}\n"
                f"Buy Trade Details: {buy_trade.to_dict()}\n"
                f"Sell Trade Details: {sell_trade.to_dict()}\n"
            )
            send_email(email_subject, email_content)

# Main loop to run the process every 10 minutes
while True:
    process_opportunities()
    time.sleep(600)  # Wait 10 minutes before the next check


Latest time: 1714232856401
Checking trades from 1714232256401 to 1714232856401
Found 20 trades in the last 10 minutes.
All Symbols - Min Buy Price: 62920.9, Max Sell Price: 63652.0, Price Difference: 731.0999999999985


IndexError: single positional indexer is out-of-bounds

### Final code 

In [ ]:
import time
import pandas as pd
from pymongo import MongoClient
import smtplib
from email.mime.text import MIMEText

# Function to send email
def send_email(subject, content):
    message = MIMEText(content)
    message['From'] = SENDER_EMAIL
    message['To'] = RECEIVER_EMAIL
    message['Subject'] = subject

    server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    server.starttls()
    server.login(SENDER_EMAIL, SENDER_PASSWORD)
    server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message.as_string())
    server.quit()

# Function to process recent data for arbitrage opportunities
def process_opportunities():
    with MongoClient(DATABASE_URL) as client:
        db = client[DATABASE_NAME]
        collection = db[COLLECTION_NAME]

        latest_document = collection.find_one({}, sort=[('time_exchange', -1)])
        if not latest_document:
            print("No trades found in the database.")
            return

        latest_time = latest_document['time_exchange']
        ten_minutes_ago = latest_time - 10 * 60 * 1000

        recent_trades = list(collection.find({'time_exchange': {'$gte': ten_minutes_ago, '$lte': latest_time}}))
        if not recent_trades:
            print("No trades found in the last 10 minutes.")
            return

        recent_data = pd.DataFrame(recent_trades)
        recent_data = recent_data[recent_data['min_buy_price'] > 0]

        if recent_data.empty:
            print("No trades with a non-zero min buy price in the last 10 minutes.")
            return

        min_buy_price = recent_data['min_buy_price'].min()
        max_sell_price = recent_data['max_sell_price'].max()
        price_diff = max_sell_price - min_buy_price

        print(f"Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}, Price Difference: {price_diff}")

        if price_diff > 0:
            buy_trades = recent_data[recent_data['min_buy_price'] == min_buy_price]
            sell_trades = recent_data[recent_data['max_sell_price'] == max_sell_price]

            if buy_trades.empty or sell_trades.empty:
                print("No matching trades found for the min and max prices.")
                return

            buy_trade = buy_trades.iloc[0]
            sell_trade = sell_trades.iloc[0]

            email_subject = "Arbitrage Opportunity Detected"
            email_content = (
                f"Min Buy Price: {min_buy_price}, Max Sell Price: {max_sell_price}\n"
                f"Price Difference: {price_diff}\n"
                f"Buy Trade Details: {buy_trade.to_dict()}\n"
                f"Sell Trade Details: {sell_trade.to_dict()}\n"
            )
            send_email(email_subject, email_content)

# Main loop to run the process every 10 minutes
while True:
    process_opportunities()
    time.sleep(600)


Min Buy Price: 62920.9, Max Sell Price: 63652.0, Price Difference: 731.0999999999985
Min Buy Price: 63225.0, Max Sell Price: 63216.0, Price Difference: -9.0
Min Buy Price: 63259.1, Max Sell Price: 63482.0, Price Difference: 222.90000000000146
